In [1]:
import pandas as pd
import numpy as np
import dask as dd
from typing import Tuple
import shortuuid

In [2]:
malcom_data_sheet = pd.ExcelFile('../Malcom Feb 17 2020.xlsx')
existing_actors = pd.read_csv('actors.csv')
existing_actors = existing_actors.apply(lambda x: x.str.lower().str.strip() if x.dtype == object else x )
existing_organizations = pd.read_csv('organizations.csv')
existing_acts = existing_organizations.apply(lambda x: x.str.lower().str.strip() if x.dtype == object else x)
relevant_sheets = ['Actors', 'Organizations', 'Positions' , 'Project']
to_lower_lists = ['Last Name',	'MI', 'First Name'	,'Name'	,'Short Name']
new_actors = {'full_name': [] , 'first_name':[] , 'last_name':[] , 'middle_initial':[] , 'act_id': []}

In [3]:
dataframe_lists = {}
for i in relevant_sheets:
    dataframe_lists[i] = pd.read_excel(malcom_data_sheet , i)

dataframe_lists['Actors'] = dataframe_lists['Actors'].apply(lambda x : x.str.lower() if x.name in to_lower_lists else x)
dataframe_lists['Actors'].dtypes


Input Source             object
Last Name                object
MI                       object
First Name               object
Name                     object
ACTID                    object
Short Name               object
Rank                     object
Branch                   object
Entered service         float64
Retired from service    float64
Service Role             object
Retired or died         float64
dtype: object

In [4]:
def create_actor(full_name:str, **kwargs)-> Tuple[bool , int]:
    first_name = str(kwargs.get('first_name' , None))
    last_name = str(kwargs.get('last_name' , None))
    middle_initial = str(kwargs.get('middle_initial' , None))
    if middle_initial is not None:
        middle_initial = middle_initial.strip('.,')

    id = shortuuid.ShortUUID().random(length=12)
    new_actors['full_name'].append(full_name)
    new_actors['first_name'].append(first_name)
    new_actors['last_name'].append(last_name)
    new_actors['middle_initial'].append(middle_initial)
    new_actors['act_id'].append(id)
    return id


def check_actor_exist(full_name:str, **kwargs)-> Tuple[bool , int]:
    actor = existing_actors.loc[existing_actors['full_name'] == full_name]
    first_name = kwargs.get('first_name' , None)
    last_name = kwargs.get('last_name' , None)
    middle_initial = kwargs.get('middle_initial' , None)
    if(actor.empty):
        if last_name != None:
            try:
                first_name.lower()
                actor = existing_actors.loc[existing_actors['last_name'] == last_name]
                if not actor.empty and first_name != None:
                    actor = actor.loc[actor['first_name'] == first_name]
                    if not actor.empty:
                        return (True , actor.get('act_id').to_numpy()[0])
                    else:
                        id = create_actor(full_name , first_name=first_name , last_name=last_name , middle_initial=middle_initial)
                        return (True, id)
                else:
                    id = create_actor(full_name , first_name=first_name , last_name=last_name , middle_initial=middle_initial)
                    return (True , id)
            except Exception as e:
                return (False , -1 , e)
    elif(actor.shape[0] > 1):
        pass
    else:
        return (True , actor.get('act_id').to_numpy()[0])    


In [5]:
def check_all_actors(actors):
    try:
        print(actors.apply(lambda x : check_actor_exist(x['Name'] , first_name=x['First Name'] , last_name=x['Last Name'] , middle_initial=x['MI']) , axis=1))
        return (True , 1)
    except Exception as e: 
        return (False , -1 , e)

check_all_actors(dataframe_lists['Actors'])
new_actors = pd.DataFrame.from_dict(new_actors)
existing_actors = pd.concat([existing_actors , new_actors])



0      (True, 7DTvuAFx3v9c)
1      (True, bPpCZ9dXCzct)
2      (True, 56AW6FCUpZF9)
3      (True, 5UmyPSgbs3AF)
4      (True, 5d7jPMmhYewm)
               ...         
798    (True, 3wdBZQjS8SsA)
799    (True, 4Urnyh9gGKX3)
800    (True, 3C5K5ybpcwnD)
801    (True, 7ug8uqnrt5Ve)
802    (True, xGQHdiBkJUAD)
Length: 803, dtype: object
(True, 1)


In [7]:
print(existing_actors.shape)
print(existing_actors.values)

(802, 6)
[['abbot s. charles' 'charles' 'abbot' nan '7DTvuAFx3v9c' 's']
 ['regni f john' 'john' 'regni' nan 'bPpCZ9dXCzct' 'f']
 ['weisman s. david' 'david' 'weisman' nan '56AW6FCUpZF9' 's']
 ...
 ['christine rake' 'rake' 'christine' nan '3C5K5ybpcwnD' 'nan']
 ['qubad talabany' 'talabany' 'qubad' nan '7ug8uqnrt5Ve' 'j. ']
 ['tom squituri' 'squituri' 'tom' nan 'xGQHdiBkJUAD' 'nan']]
